Блок по предобработке данных весьма внушительный и в данном разделе решили его опустить.

**Импорт библиотек**

In [27]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 6.1 MB/s eta 0:00:00


In [28]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 10.8 MB/s eta 0:00:00


In [29]:
import pandas as pd
import time
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LassoCV
from scipy.stats import randint
from catboost import CatBoostRegressor
import optuna
from optuna.pruners import SuccessiveHalvingPruner

##**Загрузка предобработанного датасета. Разбивка на тренировочную и тестовую выборки**

In [30]:
df = pd.read_csv('/content/data_clean.csv')

In [31]:
pd.set_option('display.max_columns', None)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12409 entries, 0 to 12408
Data columns (total 64 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   walk_to_metro_flg            12409 non-null  bool   
 1   price                        12409 non-null  float64
 2   m_sqarea                     12409 non-null  float64
 3   floor_flat                   12409 non-null  float64
 4   floor_max                    12409 non-null  float64
 5   rooms                        12409 non-null  int64  
 6   district_name                12409 non-null  object 
 7   dist_to_center               12409 non-null  float64
 8   room_square                  12409 non-null  float64
 9   Холодильник                  12409 non-null  float64
 10  Посудомоечная машина         12409 non-null  float64
 11  Стиральная машина            12409 non-null  float64
 12  Кондиционер                  12409 non-null  float64
 13  Телевизор       

In [33]:
df.head()

,walk_to_metro_flg,price,m_sqarea,floor_flat,floor_max,rooms,district_name,dist_to_center,room_square,Холодильник,Посудомоечная машина,Стиральная машина,Кондиционер,Телевизор,Интернет,Мебель на кухне,Мебель в комнатах,Оплата ЖКХ,Срок аренды,С животными,С детьми,Площадь кухни,Ремонт,Год постройки,Тип перекрытий,Парковка,Отопление,school_1000m,shops_200m,theatre_1000m,fitness_500m,cinemas_1500m,foods_1000m,kindergarten_500m,libraries_1000m,parks_2000m,post_office_1000m,climbing_1000m,fields_1000m,indoor_swimming_pool_1000m,malls_2000m,outdoor_swimming_pool_1000m,is_first_floor,is_max_floor,сумма_лифтов,грузовой_лифт,сумма_санузлов,совмещенный,сумма_балконов,балкон,park,school,shops,theatre,fitness,cinemas,foods,kindergarten,libraries,climbing,fields,outdoor_swimming_pool,indoor_swimming_pool,malls
0,False,55000.0,35.4,2.0,9.0,1,Восточный административный округ,12.0,35.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,включены в сумму,несколько месяцев,False,True,10.0,Косметический,1968.0,Железобетонные,Наземная,Центральное,19,1,1,0,0,80,2,2,12,0,0,0,0,22,0,False,False,1,False,1,True,0,False,True,True,True,True,False,False,True,True,True,False,False,False,False,True
1,False,55000.0,42.0,9.0,16.0,2,Троицкий административный округ,34.0,21.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,счетчики отдельно,от года,False,True,16.0,Евроремонт,2022.0,Железобетонные,Наземная,Индивидуальный тепловой пункт,0,6,0,0,0,3,0,0,2,0,0,0,0,0,0,False,False,2,True,1,True,1,True,True,False,True,False,False,False,True,False,False,False,False,False,False,False
2,False,50000.0,41.8,5.0,13.0,1,Троицкий административный округ,34.0,41.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,счетчики отдельно,от года,по договоренности,по договоренности,10.0,Дизайнерский,2022.0,Железобетонные,Наземная,Индивидуальный тепловой пункт,0,4,0,0,0,3,0,0,2,0,0,0,0,0,0,False,False,2,True,1,True,1,True,True,False,True,False,False,False,True,False,False,False,False,False,False,False
3,False,40000.0,40.0,11.0,14.0,1,Троицкий административный округ,34.0,40.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,счетчики отдельно,от года,по договоренности,по договоренности,10.0,Косметический,2022.0,Железобетонные,Наземная,Индивидуальный тепловой пункт,0,4,0,0,0,3,0,0,1,0,0,0,0,0,0,False,False,2,True,1,True,1,True,True,False,True,False,False,False,True,False,False,False,False,False,False,False
4,False,45000.0,42.0,13.0,14.0,1,Троицкий административный округ,34.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,счетчики отдельно,от года,по договоренности,по договоренности,7.0,Косметический,2022.0,Железобетонные,Наземная,Индивидуальный тепловой пункт,0,4,0,0,0,3,0,0,1,0,0,0,0,0,0,False,False,2,True,1,True,1,True,True,False,True,False,False,False,True,False,False,False,False,False,False,False


In [34]:
# Определение таргет и фичей
X = df.drop('price', axis=1)
y = df['price']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

# Построение моделей

## 1. Модель линейной регрессии

In [35]:
cat_features = [
    'district_name', 'Оплата ЖКХ', 'Срок аренды', 'С животными', 'С детьми',
    'Ремонт', 'Тип перекрытий', 'Парковка', 'Отопление', 'сумма_лифтов'
]

In [36]:
# кодирование категориальных признаков
encoder = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
], remainder='passthrough')

X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

In [37]:
# Применяем StandardScaler для масштабирования признаков
# Для линейной модели отмасштабируем булевые признаки тоже
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

In [38]:
# Обучение
start_time = time.time()
model = LinearRegression()
model.fit(X_train_scaled, y_train)

end_time = time.time()
training_time_lin = end_time - start_time
print(f"Время обучения: {training_time_lin:.2f} секунд")

# Предсказания
y_pred = model.predict(X_test_scaled)

# Метрики
rmse_lin = mean_squared_error(y_test, y_pred)
mae_lin = mean_absolute_error(y_test, y_pred)
r2_lin = r2_score(y_test, y_pred)
mape_lin = mean_absolute_percentage_error(y_test, y_pred) * 100

print(f"R²: {r2_lin:.4f}")
print(f"MAE: {mae_lin:.1f}")
print(f"RMSE: {rmse_lin:.1f}")
print(f"MAPE: {mape_lin:.2f}%")

Время обучения: 0.15 секунд
R²: 0.7614
MAE: 12280.1
RMSE: 297667152.1
MAPE: 15.32%


## 2. Модель регрессии Случайный лес

Построим модель с использованием pipline для примера, что так тоже можно

In [39]:
# Выделим признаки
numerical_cols = X.select_dtypes(include=['int64', 'float64', 'bool']).columns.tolist()

# Препроцессинг
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numerical_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
])

# Пайплайн
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor), #кодировка
#    ('feature_selection', SelectFromModel(LassoCV(cv=5, random_state=42))), #отброс лишних параметров
    ('model', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)) #модель
])

start_time = time.time()
# Обучение
pipeline.fit(X_train, y_train)

end_time = time.time()
training_time = end_time - start_time
print(f"Время обучения: {training_time:.2f} секунд")

# Предсказания
y_pred = pipeline.predict(X_test)

# Метрики
rmse_forest = mean_squared_error(y_test, y_pred)
mae_forest = mean_absolute_error(y_test, y_pred)
r2_forest = r2_score(y_test, y_pred)
mape_forest = mean_absolute_percentage_error(y_test, y_pred) * 100

print(f"R²: {r2_forest:.4f}")
print(f"MAE: {mae_forest:.1f}")
print(f"RMSE: {rmse_forest:.1f}")
print(f"MAPE: {mape_forest:.2f}%")

Время обучения: 27.05 секунд
R²: 0.8119
MAE: 10622.8
RMSE: 234705953.4
MAPE: 13.16%


In [40]:
# Параметры для поиска
param_distributions = {
    'model__n_estimators': randint(100, 300),
    'model__max_depth': [10, 20, 30, None],
    'model__min_samples_split': randint(2, 10),
    'model__min_samples_leaf': randint(1, 5),
    'model__max_features': ['auto', 'sqrt', 'log2']
}

# RandomizedSearchCV
search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=20,
    scoring='neg_root_mean_squared_error',
    cv=3,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

# Запуск
search.fit(X_train, y_train)

# Лучшие параметры и результат
print("Лучшие параметры:", search.best_params_)
print(f"Лучшая RMSE (на CV): {-search.best_score_:.2f}")

Fitting 3 folds for each of 20 candidates, totalling 60 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
21 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
21 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **la

Лучшие параметры: {'model__max_depth': 30, 'model__max_features': 'sqrt', 'model__min_samples_leaf': 1, 'model__min_samples_split': 3, 'model__n_estimators': 187}
Лучшая RMSE (на CV): 15508.42


In [42]:
# Пайплайн
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor), #кодировка
#    ('feature_selection', SelectFromModel(LassoCV(cv=5, random_state=42))), #отброс лишних параметров
    ('model', RandomForestRegressor(max_depth=30, min_samples_leaf=1, n_estimators=187, random_state=42, n_jobs=-1)) #модель
])

start_time = time.time()
# Обучение
pipeline.fit(X_train, y_train)

end_time = time.time()
training_time = end_time - start_time
print(f"Время обучения: {training_time:.2f} секунд")

# Предсказания
y_pred = pipeline.predict(X_test)

# Метрики
rmse_forest = mean_squared_error(y_test, y_pred)
mae_forest = mean_absolute_error(y_test, y_pred)
r2_forest = r2_score(y_test, y_pred)
mape_forest = mean_absolute_percentage_error(y_test, y_pred) * 100

print(f"R²: {r2_forest:.4f}")
print(f"MAE: {mae_forest:.2f}")
print(f"RMSE: {rmse_forest:.2f}")
print(f"MAPE: {mape_forest:.2f}%")

Время обучения: 44.45 секунд
R²: 0.8103
MAE: 10666.81
RMSE: 236684339.47
MAPE: 13.22%


## 3. Построение модели CatBoost

Для использования модели CatBoost не требуется вручную стандартизировать числовые признаки или кодировать категориальные object-столбцы, потому что:

Он автоматически обрабатывает категориальные признаки — в том числе object, bool, int, если мы передадим их в список cat_features.

Стандартизация (масштабирование) числовых признаков не нужна, т.к. CatBoost — это бустинг над деревьями, а не линейная модель.

Подберем параметры вручную

In [44]:
start_time = time.time()

model = CatBoostRegressor(
    random_state=42,
    iterations=500,
    learning_rate=0.1,
    depth=6,
    cat_features=cat_features,
    verbose=100
)
model.fit(X_train, y_train)

end_time = time.time()
training_time = end_time - start_time
print(f"Время обучения: {training_time:.2f} секунд")

y_pred = model.predict(X_test)

# Метрики
rmse_cat = mean_squared_error(y_test, y_pred)
mae_cat = mean_absolute_error(y_test, y_pred)
r2_cat = r2_score(y_test, y_pred)
mape_cat = mean_absolute_percentage_error(y_test, y_pred) * 100

print(f"R²: {r2_cat:.4f}")
print(f"MAE: {mae_cat:.2f}")
print(f"RMSE: {rmse_cat:.2f}")
print(f"MAPE: {mape_cat:.2f}%")

0:	learn: 33266.2144327	total: 62.2ms	remaining: 31s
100:	learn: 13310.5177670	total: 10.3s	remaining: 40.7s
200:	learn: 12206.2802917	total: 17.3s	remaining: 25.7s
300:	learn: 11457.5095884	total: 21.3s	remaining: 14.1s
400:	learn: 10849.5285047	total: 25.4s	remaining: 6.28s
499:	learn: 10311.9800457	total: 28.4s	remaining: 0us
Время обучения: 28.67 секунд
R²: 0.8461
MAE: 9705.94
RMSE: 191996670.38
MAPE: 11.83%


In [ ]:
# Сравним с предсказаниями на Трейн, чтобы исключить переобучение
#pred = model.predict(X_train)
#print(f'R²: {r2_score(y_train, pred):.4f}')
#print(f'MAE: {mean_absolute_error(y_train, y_pred):.1f}')

In [45]:
# Функция для оптимизации
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 500),
        'depth': trial.suggest_int('depth', 3, 8),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10),
        'cat_features': cat_features,
        'random_state': 42,
        'verbose': False
    }

    model = CatBoostRegressor(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return r2_score(y_test, y_pred)

In [48]:
# Настройка Pruner
pruner = SuccessiveHalvingPruner(
    min_resource=35,
    reduction_factor=3,
    min_early_stopping_rate=20
)

study = optuna.create_study(
    direction='maximize',
    pruner=pruner
)
study.optimize(objective, n_trials=10)

[I 2025-06-12 02:46:45,331] A new study created in memory with name: no-name-38d1caa9-f084-4413-bcc9-4fd877a9db72
[I 2025-06-12 02:46:46,875] Trial 0 finished with value: 0.837482731094648 and parameters: {'iterations': 124, 'depth': 6, 'learning_rate': 0.19737838450214756, 'l2_leaf_reg': 3.7219356611258383}. Best is trial 0 with value: 0.837482731094648.
[I 2025-06-12 02:46:50,927] Trial 1 finished with value: 0.8234211767259354 and parameters: {'iterations': 121, 'depth': 8, 'learning_rate': 0.07263062731929817, 'l2_leaf_reg': 8.634636990195116}. Best is trial 0 with value: 0.837482731094648.
[I 2025-06-12 02:46:59,535] Trial 2 finished with value: 0.8443646623846635 and parameters: {'iterations': 295, 'depth': 6, 'learning_rate': 0.2144905610506656, 'l2_leaf_reg': 0.26471751148368683}. Best is trial 2 with value: 0.8443646623846635.
[I 2025-06-12 02:47:07,280] Trial 3 finished with value: 0.8400917454131486 and parameters: {'iterations': 210, 'depth': 6, 'learning_rate': 0.157990783

In [49]:
print('Лучшие параметры:', study.best_params)
print('Лучший R2:', study.best_value)
print(f'Trials завершено: {len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])}/35')
print(f'Trials отсечено: {len([t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED])}')

Лучшие параметры: {'iterations': 295, 'depth': 6, 'learning_rate': 0.2144905610506656, 'l2_leaf_reg': 0.26471751148368683}
Лучший R2: 0.8443646623846635
Trials завершено: 10/35
Trials отсечено: 0


In [50]:
start_time = time.time()

#Обучение финальной модели
best_model = CatBoostRegressor(
    random_state=42,
    iterations=295,
    learning_rate=0.2144905610506656,
    depth=6,
    l2_leaf_reg=0.26471751148368683,
    cat_features=cat_features,
    early_stopping_rounds=20,
    verbose=False
)
best_model.fit(X_train, y_train)

end_time = time.time()
training_time = end_time - start_time
print(f"Время обучения: {training_time:.2f} секунд")

# Оценка
y_pred = best_model.predict(X_test)

# Метрики
rmse_cat_new = mean_squared_error(y_test, y_pred)
mae_cat_new = mean_absolute_error(y_test, y_pred)
r2_cat_new = r2_score(y_test, y_pred)
mape_cat_new = mean_absolute_percentage_error(y_test, y_pred) * 100

print(f"R²: {r2_cat_new:.4f}")
print(f"MAE: {mae_cat_new:.2f}")
print(f"RMSE: {rmse_cat_new:.2f}")
print(f"MAPE: {mape_cat_new:.2f}%")

Время обучения: 9.47 секунд
R²: 0.8444
MAE: 9700.67
RMSE: 194193201.98
MAPE: 11.95%


Метрики стали незначительно хуже, но все еще хорошие, зато скорость обучения сократилась в 3 раза.

# Сравнение моделей